# LLMs check

In [ ]:
from huggingface_hub import login

login("PLACE_YOUR_TOKEN")

## Quantized Mistral-7B-Instruct-v0.2

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

model_name = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quantization_config
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
prompt = """[INST] What is the NYSE tikers of these companies?
Apple, Tesla, Nvidia, Microsoft
[/INST]"""

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=100)
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(decoded_output.split("[/INST]")[-1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 The NYSE tickers for the companies you mentioned are as follows:

* Apple Inc.: AAPL
* Tesla, Inc.: TSLA
* Nvidia Corporation: NVDA
* Microsoft Corporation: MSFT


## Flan-t5-small

In [ ]:
# pip install accelerate
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small", device_map="auto")
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

input_text = "translate English to German: How old are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))


# Tools

In [ ]:
import yfinance as yf
from datetime import datetime, timedelta

def fetch_portfolio_data(tickers: list, days=30):
    """Downloads portfolio data"""
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days)

    data = yf.download(
        tickers=" ".join(tickers),
        start=start_date.strftime('%Y-%m-%d'),
        end=end_date.strftime('%Y-%m-%d'),
        group_by='ticker'
    )
    return {ticker: data[ticker] for ticker in tickers}

portfolio = ["AAPL", "MSFT", "TSLA"]
stock_data = fetch_portfolio_data(portfolio)
stock_data['AAPL'][:10]

/tmp/ipython-input-2-2656228369.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(
[*********************100%***********************]  3 of 3 completed


Price,Open,High,Low,Close,Volume
Date,,,,,
2025-05-28,200.589996,202.729996,199.899994,200.419998,45339700
2025-05-29,203.580002,203.809998,198.509995,199.949997,51396800
2025-05-30,199.369995,201.960007,196.779999,200.850006,70819900
2025-06-02,200.279999,202.130005,200.119995,201.699997,35423300
2025-06-03,201.350006,203.770004,200.960007,203.270004,46381600
2025-06-04,202.910004,206.240005,202.100006,202.820007,43604000
2025-06-05,203.500000,204.750000,200.149994,200.630005,55126100
2025-06-06,203.000000,205.699997,202.050003,203.919998,46607700
2025-06-09,204.389999,206.000000,200.020004,201.449997,72862600


In [ ]:
import pandas as pd

def analyze_trends(data: dict):
    """pandas trends analyse"""
    reports = {}
    for ticker, df in data.items():
        last_close = df['Close'][-1]
        sma_10 = df['Close'].rolling(10).mean()[-1]

        reports[ticker] = {
            "last_price": last_close.item(),
            "trend": "up ↑" if last_close > sma_10 else "down ↓",
            "volatility": df['Close'].pct_change().std()
        }
    return reports

analysis = analyze_trends(stock_data)
analysis

/tmp/ipython-input-3-108356434.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  last_close = df['Close'][-1]
/tmp/ipython-input-3-108356434.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sma_10 = df['Close'].rolling(10).mean()[-1]


{'AAPL': {'last_price': 201.0,
  'trend': 'up ↑',
  'volatility': 0.010650301843965615},
 'MSFT': {'last_price': 497.45001220703125,
  'trend': 'up ↑',
  'volatility': 0.006264411328982318},
 'TSLA': {'last_price': 325.7799987792969,
  'trend': 'down ↓',
  'volatility': 0.046404621158714035}}

In [ ]:
import feedparser

def get_financial_news(ticker: str, limit=5):
    """Parsing of RSS-feed Yahoo Finance"""
    url = f"https://feeds.finance.yahoo.com/rss/2.0/headline?s={ticker}&region=US&lang=en-US"
    news = feedparser.parse(url)
    # for item in news:
    #     print(item, news[item])
    return [
        {"title": entry.title, "timestompt": entry.published, "summary": entry.summary}
        for entry in news.entries[:limit]
    ]

aapl_news = get_financial_news("AAPL")
aapl_news

[{'title': 'Apple revamps EU App Store terms to avert more fines',
  'timestompt': 'Fri, 27 Jun 2025 10:44:04 +0000',
  'summary': "Apple has revamped its app store policies in the European Union with hopes of fending off escalating fines under the 27-nation bloc's digital competition regulations.  The bloc's executive Commission punished Apple for preventing app makers from pointing users to cheaper options outside its App Store, and gave it a 60-day deadline, which expired Thursday, to avoid additional, periodic fines.  The changes made by Apple will make it easier for app makers to point users to better deals on digital products and options to pay for them outside of Apple's own App Store, including other websites, apps or alternative app stores."},
 {'title': 'Meet the Only "Magnificent Seven" Stock That Is Cheaper Than the S&P 500 (According to This Key Metric)',
  'timestompt': 'Fri, 27 Jun 2025 10:17:00 +0000',
  'summary': 'It has a diversified business but is still heavily dep

In [ ]:
from transformers import pipeline

sentiment_analyzer = pipeline(
    "text-classification",
    model="yiyanghkust/finbert-tone",
    device="cuda" if torch.cuda.is_available() else "cpu"
)

Device set to use cuda


In [ ]:
result = sentiment_analyzer(aapl_news[4]["summary"])
label = result[0]['label']
label

'Neutral'

In [ ]:
for item in aapl_news:
    # print(item["summary tone"])
    item["summary tone"] = sentiment_analyzer(item["summary"])[0]["label"]

In [ ]:
aapl_news

[{'title': 'Alibaba Just Released New AI Models for Apple. Does That Make AAPL Stock a Buy?',
  'timestompt': 'Thu, 26 Jun 2025 19:00:02 +0000',
  'summary': 'Apple needs China, and Alibaba just helped unlock its AI future with the Qwen3 model. Is this the catalyst AAPL bulls were waiting for?',
  'summary tone': 'Positive'},
 {'title': 'Apple Stock Gets Price-Target Cut On Mounting Challenges',
  'timestompt': 'Thu, 26 Jun 2025 18:48:42 +0000',
  'summary': 'Investment bank JPMorgan on Thursday cut its price target on Apple stock, citing growing challenges for the iPhone maker.',
  'summary tone': 'Negative'},
 {'title': "Apple Continues to Expand Services Business: What's the Path Ahead?",
  'timestompt': 'Thu, 26 Jun 2025 16:42:00 +0000',
  'summary': "AAPL's Services arm hits 1B+ paid subscribers as Apple Intelligence boosts offerings from Wallet to Music.",
  'summary tone': 'Positive'},
 {'title': 'eSIM Market Size is Set to Achieve USD 17.98 Billion by 2031, With CAGR of 18.7% |

In [ ]:
def generate_advice(context: str):
    prompt = f"""[INST]
    You are finantial advisor. Based on the given data:
    Context: {context}
    Give short recommendation BUY/HOLD/SELL/SKIP and SHORT/LONG if confident. Do not explain anything [/INST]"""
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    tokenizer.pad_token = tokenizer.eos_token
    outputs = model.generate(**inputs, pad_token_id=tokenizer.eos_token_id, max_new_tokens=100, num_beams=3,do_sample=False,early_stopping=True)
    advise = tokenizer.decode(outputs[0], skip_special_tokens=True).split("[/INST]")[-1]
    return advise

context = f"""
paper: {portfolio[0]}
prices: {analysis[portfolio[0]]}
news: {aapl_news}
"""
advisor_answer = generate_advice(context)
# print(advisor_answer)
advisor_answer

" Based on the given data, my recommendation would be:\n\nBUY (Positive news outweighs negative news and overall trend is up)\n\nHowever, please note that this recommendation is based solely on the information provided and should not be considered as financial advice. It's important to conduct thorough research and consider individual circumstances before making investment decisions."